# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 16 2022 10:54AM,239749,12,8152434,Proven Skincare,Released
1,Jun 16 2022 10:50AM,239748,19,ADV80003375,"AdvaGen Pharma, Ltd",Released
2,Jun 16 2022 10:26AM,239746,12,618072,"NBTY Global, Inc.",Released
3,Jun 16 2022 10:26AM,239746,12,618073,"NBTY Global, Inc.",Released
4,Jun 16 2022 10:21AM,239742,10,PRF-31229,Bio-PRF,Released
5,Jun 16 2022 10:21AM,239742,10,PRF-31306,Bio-PRF,Released
6,Jun 16 2022 10:21AM,239742,10,PRF-31309,Bio-PRF,Released
7,Jun 16 2022 10:21AM,239742,10,PRF-31312,Bio-PRF,Released
8,Jun 16 2022 10:21AM,239742,10,PRF-31316,Bio-PRF,Released
9,Jun 16 2022 10:21AM,239742,10,PRF-31320,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
35,239727,Released,1
36,239742,Released,25
37,239746,Released,2
38,239748,Released,1
39,239749,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
239727,NaN,NaN,1.0
239742,NaN,NaN,25.0
239746,NaN,NaN,2.0
239748,NaN,NaN,1.0
239749,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239654,6.0,67.0,1.0
239659,0.0,0.0,1.0
239660,0.0,0.0,1.0
239664,0.0,1.0,0.0
239665,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239654,6,67,1
239659,0,0,1
239660,0,0,1
239664,0,1,0
239665,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239654,6,67,1
1,239659,0,0,1
2,239660,0,0,1
3,239664,0,1,0
4,239665,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239654,6,67,1
1,239659,,,1
2,239660,,,1
3,239664,,1,
4,239665,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 16 2022 10:54AM,239749,12,Proven Skincare
1,Jun 16 2022 10:50AM,239748,19,"AdvaGen Pharma, Ltd"
2,Jun 16 2022 10:26AM,239746,12,"NBTY Global, Inc."
4,Jun 16 2022 10:21AM,239742,10,Bio-PRF
29,Jun 16 2022 9:32AM,239727,10,"Amcyte Pharma, Inc."
30,Jun 16 2022 9:20AM,239726,10,ISDIN Corporation
34,Jun 16 2022 9:13AM,239725,10,Beach Products Inc
51,Jun 16 2022 9:13AM,239725,10,Beach Patient Assistance
54,Jun 16 2022 9:01AM,239721,20,"Exact-Rx, Inc."
56,Jun 16 2022 9:00AM,239720,20,"Exact-Rx, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 16 2022 10:54AM,239749,12,Proven Skincare,,,1
1,Jun 16 2022 10:50AM,239748,19,"AdvaGen Pharma, Ltd",,,1
2,Jun 16 2022 10:26AM,239746,12,"NBTY Global, Inc.",,,2
3,Jun 16 2022 10:21AM,239742,10,Bio-PRF,,,25
4,Jun 16 2022 9:32AM,239727,10,"Amcyte Pharma, Inc.",,,1
5,Jun 16 2022 9:20AM,239726,10,ISDIN Corporation,,,4
6,Jun 16 2022 9:13AM,239725,10,Beach Products Inc,,,20
7,Jun 16 2022 9:13AM,239725,10,Beach Patient Assistance,,,20
8,Jun 16 2022 9:01AM,239721,20,"Exact-Rx, Inc.",,,2
9,Jun 16 2022 9:00AM,239720,20,"Exact-Rx, Inc.",,,4


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 16 2022 10:54AM,239749,12,Proven Skincare,1,,
1,Jun 16 2022 10:50AM,239748,19,"AdvaGen Pharma, Ltd",1,,
2,Jun 16 2022 10:26AM,239746,12,"NBTY Global, Inc.",2,,
3,Jun 16 2022 10:21AM,239742,10,Bio-PRF,25,,
4,Jun 16 2022 9:32AM,239727,10,"Amcyte Pharma, Inc.",1,,
5,Jun 16 2022 9:20AM,239726,10,ISDIN Corporation,4,,
6,Jun 16 2022 9:13AM,239725,10,Beach Products Inc,20,,
7,Jun 16 2022 9:13AM,239725,10,Beach Patient Assistance,20,,
8,Jun 16 2022 9:01AM,239721,20,"Exact-Rx, Inc.",2,,
9,Jun 16 2022 9:00AM,239720,20,"Exact-Rx, Inc.",4,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 16 2022 10:54AM,239749,12,Proven Skincare,1,,
1,Jun 16 2022 10:50AM,239748,19,"AdvaGen Pharma, Ltd",1,,
2,Jun 16 2022 10:26AM,239746,12,"NBTY Global, Inc.",2,,
3,Jun 16 2022 10:21AM,239742,10,Bio-PRF,25,,
4,Jun 16 2022 9:32AM,239727,10,"Amcyte Pharma, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 16 2022 10:54AM,239749,12,Proven Skincare,1.0,NaN,NaN
1,Jun 16 2022 10:50AM,239748,19,"AdvaGen Pharma, Ltd",1.0,NaN,NaN
2,Jun 16 2022 10:26AM,239746,12,"NBTY Global, Inc.",2.0,NaN,NaN
3,Jun 16 2022 10:21AM,239742,10,Bio-PRF,25.0,NaN,NaN
4,Jun 16 2022 9:32AM,239727,10,"Amcyte Pharma, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 16 2022 10:54AM,239749,12,Proven Skincare,1.0,0.0,0.0
1,Jun 16 2022 10:50AM,239748,19,"AdvaGen Pharma, Ltd",1.0,0.0,0.0
2,Jun 16 2022 10:26AM,239746,12,"NBTY Global, Inc.",2.0,0.0,0.0
3,Jun 16 2022 10:21AM,239742,10,Bio-PRF,25.0,0.0,0.0
4,Jun 16 2022 9:32AM,239727,10,"Amcyte Pharma, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2636856,147.0,18.0,0.0
12,719202,4.0,1.0,0.0
16,239699,1.0,0.0,0.0
18,239686,1.0,0.0,0.0
19,958821,11.0,18.0,15.0
20,2157187,54.0,70.0,6.0
22,719015,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2636856,147.0,18.0,0.0
1,12,719202,4.0,1.0,0.0
2,16,239699,1.0,0.0,0.0
3,18,239686,1.0,0.0,0.0
4,19,958821,11.0,18.0,15.0
5,20,2157187,54.0,70.0,6.0
6,22,719015,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,147.0,18.0,0.0
1,12,4.0,1.0,0.0
2,16,1.0,0.0,0.0
3,18,1.0,0.0,0.0
4,19,11.0,18.0,15.0
5,20,54.0,70.0,6.0
6,22,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,147.0
1,12,Released,4.0
2,16,Released,1.0
3,18,Released,1.0
4,19,Released,11.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,18,19,20,22
Status,,,,,,,
Completed,0.0,0.0,0.0,0.0,15.0,6.0,0.0
Executing,18.0,1.0,0.0,0.0,18.0,70.0,0.0
Released,147.0,4.0,1.0,1.0,11.0,54.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,18,19,20,22
0,Completed,0.0,0.0,0.0,0.0,15.0,6.0,0.0
1,Executing,18.0,1.0,0.0,0.0,18.0,70.0,0.0
2,Released,147.0,4.0,1.0,1.0,11.0,54.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,18,19,20,22
0,Completed,0.0,0.0,0.0,0.0,15.0,6.0,0.0
1,Executing,18.0,1.0,0.0,0.0,18.0,70.0,0.0
2,Released,147.0,4.0,1.0,1.0,11.0,54.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()